In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [461]:
psp = pd.read_csv('/Users/hemingyi/Dropbox/New_York_SVI/pspnet_all.csv')
df = pd.read_csv('/Users/hemingyi/Dropbox/New_York_SVI/LineContainsPoints.csv')

street = gpd.read_file('/Users/hemingyi/Dropbox/New_York_SVI/Output/anglecalculation.shp')
route = pd.read_csv('/Users/hemingyi/Dropbox/New_York_SVI/route_clean.csv')


In [462]:
#merge with index to delete the route which zscore > 1.96
route = route[['index']]
df = route.merge(df, left_on='index', right_on='index', how='inner')

In [465]:
df = df[['index', 'count', 'points']]

In [498]:
pspvalue = psp[['wall', 'building', 'tree', 'road', 'sidewalk',
       'car', 'fence', 'railing', 'signboard', 'sky', 'person', 'skyscraper',
       'van', 'awning', 'streetlight', 'ashcan', 'ceiling', 'chair', 'bicycle',
       'grass', 'earth', 'bridge', 'minibike', 'plant', 'water', 'column',
       'sculpture', 'mountain', 'booth', 'windowpane', 'pier',
       'bulletin board', 'fountain']].values
# compact the value from psp net
def val(x):
    a = int(x['Unnamed: 0'])
    return pspvalue[a]
psp['pspval'] = psp.apply(val, axis=1)

In [501]:
#get the length of lines
street['len'] = street['geometry'].length
street['FID'] = street.index
psp = psp.merge(street, left_on='FID', right_on='FID', how='inner')
psp['val'] = psp['len'] * psp['pspval']

In [689]:
# delete point which has no value
def one(x):
    return len(x['val'])
psp['one'] = psp.apply(one, axis=1)
psp = psp[psp['one'] != 0]

In [505]:
psp = psp[['FID', 'val']]

In [213]:
vals = ['wall', 'building', 'tree', 'road', 'sidewalk',
       'car', 'fence', 'railing', 'signboard', 'sky', 'person', 'skyscraper',
       'van', 'awning', 'streetlight', 'ashcan', 'ceiling', 'chair', 'bicycle',
       'grass', 'earth', 'bridge', 'minibike', 'plant', 'water', 'column',
       'sculpture', 'mountain', 'booth', 'windowpane', 'pier',
       'bulletin board', 'fountain']

In [466]:
# the column point is str
def tolist(x):
    if len(x['points']) > 3:
        point_list = [int(x) for x in x['points'][1:-1].split(',')]
        return point_list
    else:
        return np.nan

In [467]:
%%time
df['POINTS'] = df.apply(tolist, axis=1)

CPU times: user 2.25 s, sys: 76.1 ms, total: 2.33 s
Wall time: 2.35 s


/Users/hemingyi/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [618]:
psp_point = psp['FID'].values

In [619]:
# sub = df.sample(10,random_state=1)
def only_point(x):
    retA = [i for i in x['POINTS'] if i in psp_point]
    if len(retA) == 0:
        return np.nan
    else: return retA

In [620]:
%%time
# score = np.array([])
# sub = df.sample(10,random_state=1)
df['p'] = df.apply(only_point, axis=1)

CPU times: user 2.19 s, sys: 9.12 ms, total: 2.2 s
Wall time: 2.2 s


In [621]:
df = df.dropna()

In [511]:
psp[psp['FID'] == 3959]['val'].values[0]

array([array([           nan, 5.20423373e-05, 1.49995549e-08, 6.49868873e-06,
       5.78640724e-06, 2.05388642e-07, 3.43937162e-07,            nan,
                  nan,            nan, 1.04141647e-07,            nan,
       7.19715485e-08,            nan, 4.31566141e-08,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan])], dtype=object)

In [683]:
def score_cal(x):
#     matrix = []
    a = np.zeros(33)
    for i,j in enumerate(x['p']):
#         print(j)
#         a = np.r_[a, psp[psp['FID'] == j]['val'].values[0]]
        a = np.row_stack((a,psp[psp['FID'] == j]['val'].values[0]))
#         print(a)
#         locals()['var'+ str(i)] = psp[psp['FID'] == j]['val'].values
#         if i > 0:
# #                 print(j)
#             a = np.r_[a, psp[psp['FID'] == j]['val'].values]
#             locals()['var'+ str(i - 1)] = np.r_[locals()['var'+ str(i - 1)], locals()['var'+ str(i)]]
#         else: 
#             pass              
    return (np.nanmean(a[1:], axis=0))

In [684]:
%%time

df['score'] = df.apply(score_cal, axis=1)

/Users/hemingyi/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Mean of empty slice
  app.launch_new_instance()


CPU times: user 1min 57s, sys: 399 ms, total: 1min 57s
Wall time: 1min 58s


In [686]:
np.save('count',df['count'].values)

In [688]:
np.save('var',df['score'].values)